In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

# Genera una serie temporal sintética (por ejemplo, una serie sinusoidal)
N_d = 1000
Nlenght = 100
t = np.linspace(0, Nlenght, N_d)
serie_temporal = np.sin(t) + np.random.normal(0, 0.1, N_d)

# Definición de la ventana temporal deslizante
ventana_temporal = 10  # Tamaño de la ventana
ventana_salida = 1     # Tamaño de la ventana de salida (predicción a un paso)

# Divide los datos en conjuntos de entrenamiento y prueba
train_split = N_d-100
datos_entrenamiento = serie_temporal[:train_split]
datos_prueba = serie_temporal[train_split:]
datos_forecasting = serie_temporal[train_split + ventana_temporal:]
t_prueba = t[train_split + ventana_temporal:]

# Genera secuencias y etiquetas utilizando la ventana temporal
generador_entrenamiento = TimeseriesGenerator(
    datos_entrenamiento, datos_entrenamiento, length=ventana_temporal, batch_size=32
)

generador_prueba = TimeseriesGenerator(
    datos_prueba, datos_prueba, length=ventana_temporal, batch_size=1
)

# Construye el modelo MLP
modelo = Sequential([
    Dense(64, activation='relu', input_shape=(ventana_temporal,)),
    Dense(32, activation='relu'),
    Dense(ventana_salida)
])

modelo.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Entrena el modelo
modelo.fit(generador_entrenamiento, epochs=100)

# Realiza predicciones en el conjunto de prueba
predicciones = modelo.predict(generador_prueba)
predicciones.shape == (Nlenght - ventana_temporal,)


fig, ax = plt.subplots()
ax.plot(t_prueba,datos_forecasting,'r')
ax.plot(t_prueba,predicciones,'b')
print('SME:', np.mean(np.square(datos_forecasting - predicciones)))